In [1]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import ResNet50, InceptionV3, VGG19
import random
import os
from sklearn.utils import class_weight
import numpy as np

/home/syht/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../data/train'
validation_data_dir = '../data/validation'
nb_train_samples = 989
nb_validation_samples = 800
nb_class = 24
epochs = 1
batch_size = 32

In [19]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [20]:
def clf_model(input_shape, nb_class):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_class))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    model.summary()
    
    return model

In [21]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 63789 images belonging to 23 classes.


In [22]:
train_generator.classes
class_weight = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_generator.classes),
                                                 train_generator.classes)
class_weight

array([ 1.27572897,  4.18315955, 16.9111877 , 14.44497283,  1.15511653,
        2.32086593,  1.88284778,  1.40569426,  0.45788918,  1.87394242,
        8.9755171 ,  1.7226303 , 23.90892054,  8.80455487,  0.14076206,
        6.02920605,  8.5864854 ,  0.30863953,  2.30927126, 11.09373913,
       20.24404951,  0.23056237,  4.79833007])

In [24]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 5000 images belonging to 23 classes.


In [25]:
nb_train_samples = train_generator.n
nb_class = train_generator.num_classes

In [26]:
model = clf_model(input_shape, nb_class)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [27]:
checkpointer = ModelCheckpoint('first_try.h5', save_best_only=True, save_weights_only=True)
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    epochs=epochs,
    class_weight=class_weight,
    callbacks=[checkpointer])

model.save_weights('first_try.h5')

Epoch 1/1
 123/1993 [>.............................] - ETA: 44:10 - loss: 2.3689 - acc: 0.3623

KeyboardInterrupt: 

## Transfer: VGG16

In [9]:
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

epochs = 1
batch_size = 32

input_shape = (img_width, img_height, 3)

In [10]:
def vgg16_net(input_shape, nb_class):
    # Initialize VGG16 using pre-trained weights on imagenet
    model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # use transfer learning for re-training the last layers
    # Freeze first 25 layers, so that we can retrain 26th and so on using our classes
    for layer in model.layers[:-5]:
        layer.trainable = False
        
    # Adding our new layers 
    top_layers = model.output
    top_layers = Flatten(input_shape=model.output_shape[1:])(top_layers)
    top_layers = Dense(512, activation="relu")(top_layers)
    top_layers = Dropout(0.5)(top_layers)
    top_layers = Dense(512, activation="relu")(top_layers)
    out = Dense(nb_class, activation="softmax")(top_layers)
    
    model_final = Model(input = model.input, output = out)
    
    model_final.compile(loss = "categorical_crossentropy", 
                        optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), 
                        metrics=["accuracy"])
    model_final.summary()
    
    return model_final 

In [11]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 989 images belonging to 24 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
nb_train_samples = train_generator.n
nb_validation_samples = validation_generator.n
nb_class = train_generator.num_classes

In [ ]:
model = vgg16_net(input_shape, nb_class)

55779328/58889256 [===========================>..] - ETA: 4s   

In [ ]:
checkpointer = ModelCheckpoint('vgg16.h5', save_best_only=True, save_weights_only=False)
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    epochs=epochs,
    callbacks=[checkpointer])

model.save_weights('vgg16.h5')

## Transfer: RestNet50

In [5]:
class CustomRestNet50():
    def __init__(self, prefix, input_shape, nb_class):
        self.prefix = prefix
        self.input_shape = (input_shape[0], input_shape[1], 3)
        
        # compile model
        # Initialize using pre-trained weights on imagenet
        self.res50 = ResNet50(weights='imagenet', include_top=False, input_shape=self.input_shape)

        # Freeze all layers
        for layer in self.res50.layers:
            layer.trainable = False

        # Adding our new layers 
        x = self.res50.output
        x = Flatten()(x)
        #x = Dense(512, activation="relu")(x)
        #x = Dropout(0.5)(x)
        #x = Dense(512, activation="relu")(x)
        out = Dense(nb_class, activation="softmax")(x)

        self.model = Model(input = self.res50.input, output = out)

        self.model.compile(loss = "categorical_crossentropy", 
                            optimizer = optimizers.Adam(lr=0.0001),#optimizers.SGD(lr=0.00001, momentum=0.9), 
                            metrics=["accuracy"])
        self.model.summary()
        
    def fit(self, train_dir, val_dir, epochs=1, batch_size=128):
        # training data generator
        train_datagen = ImageDataGenerator()
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            shuffle=True,
            batch_size=batch_size,
            class_mode='categorical')
        
        # testing data generator
        test_datagen = ImageDataGenerator()
        validation_generator = test_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            shuffle=True,
            batch_size=batch_size,
            class_mode='categorical')
        
        # get number of traing and testing samples
        nb_train_samples = train_generator.n
        nb_validation_samples = validation_generator.n
        
        # create model checkpoint and train
        checkpointer = ModelCheckpoint(self.prefix + '.h5', save_best_only=True, save_weights_only=False)
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size,
            epochs=epochs,
            callbacks=[checkpointer])

        # save the last weights
        self.model.save_weights(self.prefix + '_final.h5')

In [6]:
input_image = (224, 224)

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [7]:
model = CustomRestNet50('resnet50', input_shape=input_image, 24)

SyntaxError: positional argument follows keyword argument (<ipython-input-7-a5ec256eae24>, line 1)

## Transfer InceptionV3

In [3]:
input_image = (224, 224)

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [4]:
class CustomInceptionV3():
    def __init__(self, prefix, input_shape, nb_class):
        self.prefix = prefix
        self.input_shape = (input_shape[0], input_shape[1], 3)
        
        # compile model
        # Initialize using pre-trained weights on imagenet
        self.imagenet = InceptionV3(weights='imagenet', include_top=False, input_shape=self.input_shape)

        # Freeze all layers
        for layer in self.imagenet.layers:
            layer.trainable = False

        # Adding our new layers 
        x = self.imagenet.output
        x = Flatten()(x)
        #x = Dense(512, activation="relu")(x)
        #x = Dropout(0.5)(x)
        #x = Dense(512, activation="relu")(x)
        out = Dense(nb_class, activation="softmax")(x)

        self.model = Model(input = self.imagenet.input, output = out)

        self.model.compile(loss = "categorical_crossentropy", 
                            optimizer = optimizers.Adam(lr=0.0001),#optimizers.SGD(lr=0.00001, momentum=0.9), 
                            metrics=["accuracy"])
        self.model.summary()
        
    def fit(self, train_dir, val_dir, epochs=1, batch_size=128):
        # training data generator
        train_datagen = ImageDataGenerator()
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            shuffle=True,
            batch_size=batch_size,
            class_mode='categorical')
        
        # testing data generator
        test_datagen = ImageDataGenerator()
        validation_generator = test_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.input_shape[0], self.input_shape[1]),
            shuffle=True,
            batch_size=batch_size,
            class_mode='categorical')
        
        # get number of traing and testing samples
        nb_train_samples = train_generator.n
        nb_validation_samples = validation_generator.n
        
        # create model checkpoint and train
        checkpointer = ModelCheckpoint(self.prefix + '.h5', save_best_only=True, save_weights_only=False)
        self.model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size,
            epochs=epochs,
            callbacks=[checkpointer])

        # save the last weights
        self.model.save_weights(self.prefix + '_final.h5')

In [5]:
model = CustomInceptionV3('inceptionv3', input_shape=input_image, nb_class=24)

 6914048/87910968 [=>............................] - ETA: 1:21 

KeyboardInterrupt: 